In [264]:
### CBC SCRAPING CODE
### Authors: JONATHAN CHAN and PANDRAMISHI NAGA SIRISHA

###MOST RECENT UPDATE:  
##2020 MAY 13, 11:52AM
#wrote separate functions for each JSON element to be collected
#extract_json_items() will run for all articles, and will return null if not in proper format

#TO DO:
#write code for collecting JSON items from all articles returned in CBC API (code cell 4)
#write code to iterate through multiple pages of API call 
#done
#write code to store JSON in a JSON or text document
#Final run: collect 50-100 articles for each of 6 indicators

In [265]:
import urllib.request
import json 
from bs4 import BeautifulSoup
#from datetime import date
import requests
import json
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import datetime, timedelta
import datetime
import pytz
import dateutil.parser


In [266]:
#https://www.cbc.ca/search_api/v1/search?q=mortgage%20rate&sortOrder=relevance&page=100&fields=feed
def get_initial_url(search_term):
    """returns the URL of the first page API call given a search string"""
    
    words = search_term.split()
    url_prefix = "https://www.cbc.ca/search_api/v1/search?"
    query = "q=" + "%20".join(words)
    url_suffix = "&sortOrder=relevance&page=1&fields=feed"
    first_url = url_prefix + query + url_suffix
    print("FIRST URL API CALL: ", first_url)
    return first_url
    
get_initial_url("interest rate index")

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed


'https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed'

In [267]:

my_date = datetime.datetime.now(datetime.timezone.utc)
past = my_date - timedelta(days=365)
print(past)

2019-05-16 03:40:24.680927+00:00


In [268]:
yourdate = dateutil.parser.parse('2020-05-13T14:48:05.000Z')
print(yourdate)
yourdate > past

2020-05-13 14:48:05+00:00


True

In [ ]:
def scrape_urls(url):
    """This function takes in the first query url and scrapes all other articles from past 1 year and returns 
    the urls of such articles"""
    count = 0
    url_list = []
    current_date = datetime.datetime.now(datetime.timezone.utc)
    last_year_date = current_date - timedelta(days=365)
    main_url = url
    r = requests.get(url)
    info = r.json()
    last_retrieved_items_count= len(info)
    
    for i in info:
        count += 1
        if dateutil.parser.parse(i['publishtime']) > last_year_date:
            url_list.append(i['url'])
#             print(i['publishtime'])
        
    page_number = 2
    
#     while last_retrieved_items_count > 0 :
    while page_number < 25 :

        split_url = main_url.split('page')
        new_url = split_url[0] + "page=" + str(page_number) + "&fields=feed" 
        r = requests.get(new_url)
        info = r.json()
        last_retrieved_items_count= len(info)
#         print("last_retrieved_items_count", last_retrieved_items_count)

        for i in info:
            count += 1
            if dateutil.parser.parse(i['publishtime']) > last_year_date:
                url_list.append(i['url'])
#                 print(i['publishtime'])
            
        page_number += 1    
#         print(count)
    return url_list

# first_url = get_initial_url("supernova")
# all_urls = scrape_urls(first_url)

In [293]:
def get_author(soup):
    """returns the author of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume author info is contained within span tag (class: authorText)
    """
    author_span = soup.find("span", {"class": "authorText"})
    
    if author_span:
        return author_span.text
    else:
        #print("No author found in article!")
        return None

In [294]:
def get_title(soup):
    """returns the title of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume title info is contained within h1 tag (class: detailHeadline)
    """
    title_tag = soup.find("h1", {"class": "detailHeadline"})
    
    if title_tag:
        title_text = title_tag.text
        return title_text
    else:
        #print("no title found in article!")
        return None

In [295]:
def get_desc(soup):
    """returns the description of a BeautifulSoup article if it exists, None if not
    
    Assume description is contained within h2 tag (class: deck)
    """
    desc_tag = soup.find("h2", {"class": "deck"})
    
    if desc_tag:
        desc_text = desc_tag.text
        return desc_text
    else:
        #print("No description found in article!")
        return None
    

In [296]:
def get_url_to_image(soup):
    """returns the url to the header image of a CBC article (BeautifulSoup) if it exists, None if not
    
    Assume image url is contained within src attribute of img tag 
    """
    main_image_tag = soup.find("figure", {"class": "imageMedia leadmedia-story full"})
    
    if main_image_tag:
        main_image_url = main_image_tag.find("img").attrs["src"]
        return main_image_url
    else:
        #print("No main header image found in article!")
        return None
        

In [297]:
def get_publish_time(soup):
    """returns a tuple of publish time string and datetime string if found in article, None if not
    
    Assume time is contained within time tag (class: timestamp)
    """
    time_tag = soup.find("time", {"class": "timeStamp"})
    if time_tag:
        datetime_str = time_tag.attrs["datetime"]
        
        #NOTE: if we want to return a datetime object, error when writing to JSON
        #datetime_obj = parser.isoparse(datetime_str)
        #SOLUTION: return as string for now, convert to datetime object later in pipeline
        
        #format of time_tag.text: 
        timetext_str = time_tag.text.split("|")[0].replace("Posted: ", "").strip()
        return (timetext_str, datetime_str)
    else:
        #print("No time information found in article!")
        return None


In [298]:
def get_source(soup, specify_source_type=True):
    """Returns the source of the article if it exists
    if specify_source_type, subdivision of CBC will be returned
    if not, "CBC" will be returned as the source
    
    
    Assume that source always starts with "CBC" (Ex: "CBC news", "CBC radio")
    Assume that source comes before span tag (class: bullet)
    """
    
    #source appears before <span class="bullet"> · </span>
    #if author is attached, there are two bullet tags
    #if no author attached, there is one bullet tag
    source = None
    
    if specify_source_type:
        bullet_spans = soup.find_all("span", {"class": "bullet"})
        for bullet_span in bullet_spans:
            previous_str = str(bullet_span.previous_sibling)
            if previous_str.startswith("CBC"):
                source = previous_str
    else:
        
        source = "CBC"
    
    if source:
        return source
    else:
        #print("no source found in article!")
        return None
    

In [299]:

def get_content(soup, as_string=True):
    """Returns the text content from a CBC article (as BeautifulSoup object)
    if as_string is True, return content as one string,
    if as_string is False, return content as list of paragraph strings
    
    Input: BeautifulSoup object, boolean
    
    """
    
    story_tag = soup.find("div", {"class": "story"}) 
    content_list = []
    
    if story_tag:
        for p_tag in story_tag.find_all("p"):
            p_text = p_tag.text + "\n"
            content_list.append(p_text)

        if as_string:
            final_content = "".join(content_list)
        else:
            final_content = content_list #return content as list of paragraph strings

        return final_content
    else:
        #print("no content found in article!")
        return None
    
# get_content(soup)

In [300]:
#NEW - USING NEW FUNCTIONS: 
def extract_json_items(url, specify_source_type=True):
    """Returns a json containing the following items from a CBC article:
        url: the url of the article
        urlToImage: the url of the header image
        title: the title of the article 
        description: subheader of the article
        author: author (note that some articles do not specify author)
        source: CBC if specify_source_type == False, subdivision of CBC if True (ex: "CBC radio")
        publishedAt: tuple of (date_string, datetime object)
        
        input: url returned from CBC API in "url" field (missing "http:" as part of URL)
    """
    json_dict = {}
#     output_list = [] 
    article_url = "http:" + url
    
    #get HTML from article URL into BeautifulSoup
    try:
        html_bytes = urllib.request.urlopen(article_url)
  
    except HTTPError as e:
        print('Error code: ', e.code)
        return None
    except URLError as e:
        print('Reason: ', e.reason)
        return None

    else:    
        mybytes = html_bytes.read()
        html = mybytes.decode("utf8")
        html_bytes.close()
        soup = BeautifulSoup(html, 'html.parser')
        author_name = get_author(soup)
        title_text = get_title(soup)
        desc_text = get_desc(soup)
        image_url = get_url_to_image(soup)
        publish_time = get_publish_time(soup)
        news_source = get_source(soup)
        content = get_content(soup, True)
        
        json_dict["author"] = author_name
        json_dict["title"] = title_text 
        json_dict["description"] = desc_text
        json_dict["url"] = article_url
        json_dict["urlToImage"] = image_url
        json_dict["publishedAt"] = publish_time
        json_dict["source"] = news_source
        json_dict["content"] = content
#         print("JSON KEYS AND VALUES: ")
#         for key, item in json_dict.items():
#             print(key + ": " + str(item))
#         print("-----")
        
        final_json = json.dumps(json_dict)
        return json_dict


In [301]:
extract_json_items('//www.cbc.ca/news/business/powel-trump-negative-rates-1.5567512')

{'author': 'Pete Evans',
 'title': 'U.S. Fed chair rules out negative interest rates even as Trump trumpets them',
 'description': 'U.S. president goes negative on Jerome Powell for rejection of below-zero interest rates',
 'url': 'http://www.cbc.ca/news/business/powel-trump-negative-rates-1.5567512',
 'urlToImage': 'https://i.cbc.ca/1.5258204.1566589338!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/869482910.jpg',
 'publishedAt': ('May 13, 2020 10:48 AM ET', '2020-05-13T15:17:41.088Z'),
 'source': 'CBC News',
 'content': 'The head of the Federal Reserve said Wednesday the U.S. central bank is not considering using negative interest rates, despite President Donald Trump seemingly pushing for them.\nIn a tweet Tuesday night, the president said other countries are enjoying the advantages of negative interest rates, and he urged his own central bank to accept the "gift" they would bestow on the U.S. economy.\nTypically central banks tinker with lending rates in an attempt to con

In [302]:
def main(query):
    first_url = get_initial_url(query)
    all_urls = scrape_urls(first_url)
    json_list = []
    
    for each_url in all_urls:
        retrieved_json  = extract_json_items(each_url)
        if retrieved_json is not None:
            print(each_url)
            json_list.append(retrieved_json)
    
    full_query = query.split(" ")
    file_name_prefix = "_".join(full_query)
    print(file_name_prefix)
    
    with open( file_name_prefix + '_' +'CBC_article' + '.json', 'w') as json_file:
        json.dump(json_list, json_file)
    
    return json_list



## Mortgage Rates

In [ ]:
cbc_mr_article = main("mortgage rates")
print(len(cbc_mr_article))
print(cbc_mr_article[0])

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=mortgage%20rates&sortOrder=relevance&page=1&fields=feed
//www.cbc.ca/news/business/bank-of-canada-thursday-1.5569391
//www.cbc.ca/news/business/interest-rates-coronavirus-evans-1.5515884
//www.cbc.ca/news/business/coronavirus-mortgage-rates-canada-1.5443071
//www.cbc.ca/news/business/powel-trump-negative-rates-1.5567512
//www.cbc.ca/news/canada/calgary/alberta-mortgages-arrears-august-report-from-february-2019-data-1.5246010
//www.cbc.ca/news/business/canadian-mortgage-rates-analysis-1.5164774
//www.cbc.ca/news/canada/hamilton/hamilton-economy-coronavirus-1.5566804
//www.cbc.ca/news/business/housing-starts-april-1.5561214
//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768
//www.cbc.ca/news/canada/british-columbia/covid-19-may-real-estate-1.5562836
//www.cbc.ca/news/canada/newfoundland-labrador/vianne-timmons-pov-women-1.5559214
//www.cbc.ca/news/business/canada-jobs-april-1.5561001
//www.cbc.ca/new

In [263]:
print(len(cbc_mr_article))

NameError: name 'cbc_mr_article' is not defined

## Interest rates

In [ ]:
cbc_hp_article =  main("housing price")
print(len(cbc_hp_article))
print(cbc_hp_article[0])

## Employment

In [ ]:
cbc_e_article = main('employment')
print(len(cbc_e_article))
print(cbc_e_article[0])

## GDP

In [ ]:
cbc_gdp_article = main('GDP')
print(len(cbc_gdp_article))
print(cbc_gdp_article[0])

## Stock Market

In [ ]:
cbc_tsx_article = bnn_article_scraper('stock market')
print(len(cbc_tsx_article))
print(cbc_tsx_article[0])

In [151]:
#TEST ON DIFFERENT CBC LINKS RETURNED BY CBC API

standard_url = "//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"
radio_url = "//www.cbc.ca/radio/costofliving/slashed-interest-rates-getting-a-piece-of-the-electric-car-pie-and-a-happy-jobs-friday-to-all-1.5486253"
media_url = "//www.cbc.ca/player/play/1707317315674"
noauthor_url = "//www.cbc.ca/news/canada/coronavirus-covid19-world-canada-may12-1.5564261"



extract_json_items(standard_url)

#note: doesn't work for 'player' URLS ("//www.cbc.ca/player/play/1707317315674")
#will run, but will return null for most values - player articles have different setup


'{"author": "John Paul Tasker", "title": "Federal deficit likely to be higher than $252 billion, parliamentary budget\\u00a0officer says", "description": "PBO says it\'s possible federal debt will hit $1 trillion because of pandemic relief spending", "url": "http://www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768", "urlToImage": "https://i.cbc.ca/1.5138746.1558027091!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/phoenix-pay-system.jpg", "publishedAt": ["May 12, 2020 5:34 PM ET", "2020-05-13T17:44:54.995Z"], "source": "CBC News"}'